Mock run
----
Here is the code to run the various mocks, with zeus sampler. Look at the options available to select the specific mock, with the specific inference setup you are interested into. This notebook is inspired from the notebooks in [this github page](https://github.com/TDCOSMO/hierarchy_analysis_2020_public/tree/master),from S. Birrer.

In [ ]:
import numpy as np
import os
import time
import copy
import corner
import astropy.io.fits as pyfits
import pickle, h5py
import emcee, zeus
import os
from os.path import dirname, abspath
from lenstronomy.LensModel.Profiles.nfw import NFW
from lenstronomy.LensModel.Profiles.pemd import PEMD
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.Cosmo.lens_cosmo import LensCosmo
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver
from lenstronomy.LightModel.light_model import LightModel
from lenstronomy.PointSource.point_source import PointSource
from lenstronomy.ImSim.image_model import ImageModel
import lenstronomy.Util.param_util as param_util
import lenstronomy.Util.simulation_util as sim_util
import lenstronomy.Util.image_util as image_util
import lenstronomy.Util.constants as const
from lenstronomy.Util import kernel_util
from lenstronomy.Data.imaging_data import ImageData
from lenstronomy.Data.psf import PSF
from scipy.special import gamma as gamma_func
from scipy.optimize import fsolve
from schwimmbad import MPIPool
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import sys

flag_cluster = True # Avoid plotting stuff if in cluster
mpi = False
# Constants in SI
G_const = 6.67 * 10**(-11)
arcsec2rad = 4.84814 * 10**(-6)
m_sun = 1.989 * 10**(30)
pc2meter = 3.086 * 10**(16)

np.random.seed(seed=int(time.time()))

# Mock setup, elliptical NFW or not
flag_mock = 'ellipt'
#flag_mock = 'no_ellipt'

# Inference setup, no ellipticity in the inferred NFW or same model list as in the true mock
flag_inference = 'inference_no_ellipt'
#flag_inference = 'same' # Same model on the mock and on the inference

# Prior on center on the inference; displaced center priors correspond to the ones shown in Fig. 4
#flag_center = 'center_truth'
#flag_center = 'center_displaced'
#flag_center = 'center_displaced1' # Gaussian center on the upper minimum
flag_center = 'center_displaced2' # Gaussian center on the lower left minimum
#flag_center = 'flat_prior'
#flag_center = 'no_priors' # no Priors, just the flat H0 one

# Prior on kinematics, _cprior_ has both kinematics and c_vir prior, _no_kin_ has no prior on kinematics and c_vir, 
# _ has only kinematics prior
#flag_kin = '_no_kin_'
flag_kin = '_'
#flag_kin = '_cprior_'

# sigma_low corresponds to a prior on kinematics with 60km/s as sigma
flag_sigma_uncertainty = ''
#flag_sigma_uncertainty = 'sigma_low'

flag_wide_walkers = '_' # Meaning with no wide walkers
#flag_wide_walkers = '_wide_walkers_'
#flag_wide_walkers = '_wrong_center_'
#flag_wide_walkers = '_wide_walkers_'
#flag_wide_walkers = '_scan' # Scan over some region of the walkers
if flag_wide_walkers == '_scan':
    num_scan = int(sys.argv[1])

if flag_wide_walkers == '_scan':
    name_flag='mock_files/pg1115_zeus_new'+flag_kin+flag_mock+flag_inference+flag_wide_walkers+str(num_scan)+'_'+flag_center
else:
    name_flag='mock_files/pg1115_zeus_new'+flag_kin+flag_mock+flag_inference+flag_wide_walkers+flag_center

print(name_flag)
backup_filename = name_flag+'mock_results.h5'
# Set to true if you want to continue a chain
start_from_backup= True
# Set to False if you have already run it
run_sim = True
reprocess_corner = True # Set to true if you want to reprocess the data entering the corner plot

backup_filename = name_flag+'mock_results2.h5' # This is where it stores the new chain
backup_filename2 = name_flag+'mock_results.h5' # This is where it takes the first sample if start_from_backup == True

# define lens configuration and cosmology (not for lens modelling)
z_lens = 0.311
z_source = 1.722
Om = 0.3
OL = 0.7
H0 = 67.4
sigma_halo_mean = 440 # in km/s
if flag_sigma_uncertainty == 'sigma_low':
    sigma_halo_err = 60 # in km/s
else:
    sigma_halo_err = 120 # in km/s

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=H0, Om0=Om, Ob0=0.)
lens_cosmo = LensCosmo(z_lens = z_lens, z_source = z_source, cosmo = cosmo)
dd = lens_cosmo.dd
D_source = lens_cosmo.ds
D_ds = lens_cosmo.dds
sigma_crit = lens_cosmo.sigma_crit

# data specifics
sigma_bkg = .05  #  background noise per pixel (Gaussian)
exp_time = 100.  #  exposure time (arbitrary units, flux per pixel is in units # photons/exp_time unit)
numPix = 100  #  cutout pixel size
deltaPix = 0.05  #  pixel size in arcsec (area per pixel = deltaPix**2)
fwhm = 0.1  # full width half max of PSF (only valid when psf_type='gaussian')
psf_type = 'GAUSSIAN'  # 'GAUSSIAN', 'PIXEL', 'NONE'; point spread function
#  kernel_size = 91

# initial input simulation
# generate the coordinate grid and image properties
kwargs_data = sim_util.data_configure_simple(numPix, deltaPix, exp_time, sigma_bkg)
data_class = ImageData(**kwargs_data)

kwargs_psf = {'psf_type': psf_type, 'pixel_size': deltaPix, 'fwhm': fwhm}
psf_class = PSF(**kwargs_psf)    
    
    

In [ ]:
# Lensing Modelling
# lensing quantities
theta_E_true, gamma_true = 1.08, 2.17
e1true, e2true = -0.2, 0.05
kwargs_pemd = {'theta_E': theta_E_true, 'gamma': gamma_true, 'center_x': 0.0, 'center_y': 0.0, 'e1': e1true, 'e2': e2true}  # parameters of the deflector lens model
gamma1_true_shear, gamma2_true_shear = 0.05, -0.02
kwargs_shear = {'gamma1': gamma1_true_shear, 'gamma2': gamma2_true_shear}  # shear values to the source plane

# kwargs halo
Rs_angle, alpha_Rs = lens_cosmo.nfw_physical2angle(M=2*10**14, c=3.5)
rho0_true, Rs_true, c_true, r200_true, M200_true = lens_cosmo.nfw_angle2physical(Rs_angle=Rs_angle, alpha_Rs=alpha_Rs)
theta_E_halo_true, gamma_halo_PL = 3.1, 1.5
tilde_rho_true = alpha_Rs / (4 * Rs_angle ** 2 * (1 + np.log(1. / 2.)))

print('Rs angle',Rs_angle, 'alpha_Rs angle', alpha_Rs)
x_nfw = 18.
y_nfw = -10.
e1_nfw_true, e2_nfw_true = -0.07, 0.03
if flag_mock=='ellipt':
    kwargs_nfw_halo = {'Rs': Rs_angle, 'alpha_Rs': alpha_Rs, 'center_x': x_nfw, 'center_y': y_nfw, 'e1':-0.07, 'e2':0.03}  # parameters of the deflector lens model
    lens_model_list = ['PEMD', 'SHEAR', 'NFW_ELLIPSE']
elif flag_mock == 'no_ellipt':
    kwargs_nfw_halo = {'Rs': Rs_angle, 'alpha_Rs': alpha_Rs, 'center_x': x_nfw, 'center_y': y_nfw }  # parameters of the deflector lens model
    lens_model_list = ['PEMD', 'SHEAR', 'NFW']

kwargs_lens = [kwargs_pemd, kwargs_shear, kwargs_nfw_halo]
lens_model_class = LensModel(lens_model_list=lens_model_list, z_lens=z_lens, z_source=z_source, cosmo=cosmo)

# lens light model
r_eff = 0.8
lens_light_model_list = ['HERNQUIST']
lens_light_model_class = LightModel(light_model_list=lens_light_model_list)
kwargs_hernquist = {'amp': 4000, 'Rs': r_eff*0.551, 'center_x': 0, 'center_y': 0}
kwargs_lens_light = [kwargs_hernquist]

# choice of source type
source_type = 'SERSIC'  # 'SERSIC' or 'SHAPELETS'

if flag_mock == 'ellipt':
    source_x, source_y = 5.9, -2.61
elif flag_mock == 'no_ellipt':
    source_x, source_y = 5.17, -2.8

# Sersic parameters in the initial simulation
phi_G, q = 0.3, 0.8
e1, e2 = param_util.phi_q2_ellipticity(phi_G, q)
print('e1 e2 light model', e1,e2)
kwargs_sersic_source = {'amp': 2000, 'R_sersic': 0.1, 'n_sersic': 1, 'e1': e1, 'e2': e2, 'center_x': source_x, 'center_y': source_y}
source_model_list = ['SERSIC_ELLIPSE']
kwargs_source = [kwargs_sersic_source]
source_model_class = LightModel(light_model_list=source_model_list)


lensEquationSolver = LensEquationSolver(lens_model_class)
x_image, y_image = lensEquationSolver.findBrightImage(source_x, source_y, kwargs_lens, numImages=4,
                                                      min_distance=deltaPix, search_window=numPix * deltaPix)
print(len(x_image))

mag = lens_model_class.magnification(x_image, y_image, kwargs=kwargs_lens)
kwargs_ps = [{'ra_image': x_image, 'dec_image': y_image,
                           'point_amp': np.abs(mag)*1000}]  # quasar point source position in the source plane and intrinsic brightness
point_source_list = ['LENSED_POSITION']
point_source_class = PointSource(point_source_type_list=point_source_list, fixed_magnification_list=[False])

kwargs_numerics = {'supersampling_factor': 1}

imageModel = ImageModel(data_class, psf_class, lens_model_class, source_model_class,
                                lens_light_model_class, point_source_class, kwargs_numerics=kwargs_numerics)

# Generate image
image_sim = imageModel.image(kwargs_lens, kwargs_source, kwargs_lens_light, kwargs_ps)
poisson = image_util.add_poisson(image_sim, exp_time=exp_time)
bkg = image_util.add_background(image_sim, sigma_bkd=sigma_bkg)
image_sim = image_sim + bkg + poisson

data_class.update_data(image_sim)
kwargs_data['image_data'] = image_sim

kwargs_model = {'lens_model_list': lens_model_list,
                 'lens_light_model_list': lens_light_model_list,
                 'source_light_model_list': source_model_list,
                'point_source_model_list': point_source_list
                 }

print('True parameters used', kwargs_lens, kwargs_source, kwargs_lens_light, kwargs_ps)

# display the initial simulated image
if flag_cluster == False:
    cmap_string = 'gray'
    cmap = plt.get_cmap(cmap_string)
    cmap.set_bad(color='k', alpha=1.)
    cmap.set_under('k')
    v_min = -4
    v_max = 2

    f, axes = plt.subplots(1, 1, figsize=(6, 6), sharex=False, sharey=False)
    ax = axes
    im = ax.matshow(np.log10(image_sim), origin='lower', vmin=v_min, vmax=v_max, cmap=cmap, extent=[0, 1, 0, 1])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.autoscale(False)
    plt.show()

In [ ]:
# Time delays
from lenstronomy.Analysis.td_cosmography import TDCosmography
td_cosmo = TDCosmography(z_lens, z_source, kwargs_model, cosmo_fiducial=cosmo)
# time delays, the unit [days] is matched when the lensing angles are in arcsec
t_days = td_cosmo.time_delays(kwargs_lens, kwargs_ps, kappa_ext=0)
print("the time delays for the images at position ", kwargs_ps[0]['ra_image'], kwargs_ps[0]['dec_image'], "are: ", t_days)

# relative delays (observable). The convention is relative to the first image
dt_days =  t_days[1:] - t_days[0]
dt_sigma = [2, 2, 2]  # Gaussian errors

print('Real time delays', dt_days)
# Realisation of the measurement with the quoted error bars
dt_measured = np.random.normal(dt_days, dt_sigma)
print("the measured relative delays are: ", dt_measured)

In [ ]:
# MCMC run
# lens model choicers: initial guess and upper-lower bounds
fixed_lens = []
kwargs_lens_init = []
kwargs_lens_sigma = []
kwargs_lower_lens = []
kwargs_upper_lens = []

## Power law model
fixed_lens.append({'center_x': 0.0, 'center_y': 0})
kwargs_lens_init.append(kwargs_lens[0])
kwargs_lens_sigma.append({'theta_E': .5, 'e1': 0.1, 'e2': 0.1, 'gamma': 0.2})
kwargs_lower_lens.append({'theta_E': 0.01, 'e1': -0.5, 'e2': -0.5, 'gamma': 1.5})
kwargs_upper_lens.append({'theta_E': 10, 'e1': 0.5, 'e2': 0.5, 'gamma': 2.5})

## SHEAR model
fixed_lens.append({'ra_0': 0, 'dec_0': 0})
if flag_wide_walkers == '_' or flag_wide_walkers== '_scan':
    #kwargs_lens_init.append({'gamma1': gamma1_true_shear, 'gamma2': gamma2_true_shear})
    #kwargs_lens_sigma.append({'gamma1': 0.05, 'gamma2': 0.05})
    kwargs_lens_init.append({'gamma1': 0, 'gamma2': 0})
    kwargs_lens_sigma.append({'gamma1': 0.1, 'gamma2': 0.1})
else:
    kwargs_lens_init.append({'gamma1': 0, 'gamma2': 0})
    kwargs_lens_sigma.append({'gamma1': 0.2, 'gamma2': 0.2})
kwargs_lower_lens.append({'gamma1': -0.5, 'gamma2': -0.5})
kwargs_upper_lens.append({'gamma1': 0.5, 'gamma2': 0.5})

## Nfw model
if flag_center =='center_displaced':
    x_halo_prior = 6.5
    y_halo_prior = -5
elif flag_center =='center_displaced1':
    x_halo_prior = 0
    y_halo_prior = 13
elif flag_center =='center_displaced2':
    x_halo_prior = -12
    y_halo_prior = -10
elif flag_center =='center_truth' or flag_center=='flat_prior' or flag_center =='no_priors':
    x_halo_prior = x_nfw
    y_halo_prior = y_nfw
if flag_mock == 'no_ellipt' or flag_inference =='inference_no_ellipt':
    fixed_lens.append({})
    if flag_wide_walkers == '_':
        kwargs_lens_init.append({'Rs': Rs_angle, 'alpha_Rs': alpha_Rs, 'center_x': x_halo_prior, 'center_y': y_halo_prior})
        kwargs_lens_sigma.append({'Rs': 20, 'alpha_Rs': 2, 'center_x': 16, 'center_y': 16})
    elif flag_wide_walkers == '_wide_walkers_':
        kwargs_lens_init.append({'Rs': Rs_angle, 'alpha_Rs': alpha_Rs, 'center_x': 0, 'center_y': 0})
        kwargs_lens_sigma.append({'Rs': 10, 'alpha_Rs': 1, 'center_x': 20, 'center_y': 20})

    elif flag_wide_walkers == '_scan':
        x = np.array([-20,-7,7,20])
        y = np.array([-20,-7,7,20])
        x_walker, y_walker = np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])[num_scan]
        kwargs_lens_init.append({'Rs': Rs_angle, 'alpha_Rs': alpha_Rs, 'center_x': x_walker, 'center_y': y_walker})
        kwargs_lens_sigma.append({'Rs': 10, 'alpha_Rs': 1, 'center_x': 5, 'center_y': 5})

    kwargs_lower_lens.append({'Rs': 5, 'alpha_Rs': 0.1, 'center_x': -100, 'center_y': -100.})
    kwargs_upper_lens.append({'Rs': 150, 'alpha_Rs': 15, 'center_x': 100, 'center_y': 100.})
    lens_params = [kwargs_lens_init, kwargs_lens_sigma, fixed_lens, kwargs_lower_lens, kwargs_upper_lens]

elif flag_mock == 'ellipt' and flag_inference !='inference_no_ellipt':
    fixed_lens.append({})
    kwargs_lens_init.append({'Rs': Rs_angle, 'alpha_Rs': alpha_Rs, 'center_x': x_halo_prior, 'center_y': y_halo_prior, 'e1': 0, 'e2': 0.})
    kwargs_lens_sigma.append({'Rs': 20, 'alpha_Rs': 2, 'center_x': 16, 'center_y': 16., 'e1': 0.05, 'e2': 0.05})
    kwargs_lower_lens.append({'Rs': 5, 'alpha_Rs': 0.1, 'center_x': -100, 'center_y': -100., 'e1': -0.5, 'e2': -0.5})
    kwargs_upper_lens.append({'Rs': 150, 'alpha_Rs': 15, 'center_x': 100, 'center_y': 100., 'e1': 0.5, 'e2': 0.5})
    lens_params = [kwargs_lens_init, kwargs_lens_sigma, fixed_lens, kwargs_lower_lens, kwargs_upper_lens]

# lens light model choices
fixed_lens_light = []
kwargs_lens_light_init = []
kwargs_lens_light_sigma = []
kwargs_lower_lens_light = []
kwargs_upper_lens_light = []

fixed_lens_light.append(kwargs_hernquist)
kwargs_lens_light_init.append(kwargs_hernquist)
kwargs_lens_light_sigma.append({'Rs': 0.1, 'center_x': 0.1, 'center_y': 0.1})
kwargs_lower_lens_light.append({'Rs': 0, 'center_x': -10, 'center_y': -10})
kwargs_upper_lens_light.append({'Rs': 10, 'center_x': 10, 'center_y': 10})

lens_light_params = [kwargs_lens_light_init, kwargs_lens_light_sigma, fixed_lens_light, kwargs_lower_lens_light, kwargs_upper_lens_light]

# Source choices
fixed_source = []
kwargs_source_init = []
kwargs_source_sigma = []
kwargs_lower_source = []
kwargs_upper_source = []

fixed_source.append({'amp': 2000,'n_sersic': 1, 'e1': e1, 'e2': e2})
if flag_wide_walkers == '_scan':
    kwargs_source_init.append({'R_sersic': 0.1, 'center_x': 0, 'center_y': 0})
    kwargs_source_sigma.append({'R_sersic': 0.02, 'center_x': 2.5, 'center_y': 2.5})
else:
    kwargs_source_init.append({'R_sersic': 0.1, 'center_x': 0, 'center_y': 0})
    kwargs_source_sigma.append({'R_sersic': 0.02, 'center_x': 10, 'center_y': 10})
kwargs_lower_source.append({'R_sersic': 0.01, 'center_x': -100, 'center_y': -100})
kwargs_upper_source.append({'R_sersic': 0.3, 'center_x': 100, 'center_y': 100})

source_params = [kwargs_source_init, kwargs_source_sigma, fixed_source, kwargs_lower_source, kwargs_upper_source]

# Point source model
fixed_ps = [{}]
kwargs_ps_init = kwargs_ps
kwargs_ps_sigma = [{'ra_image': 0.01 * np.ones(len(x_image)), 'dec_image': 0.01 * np.ones(len(x_image))}]
kwargs_lower_ps = [{'ra_image': -10 * np.ones(len(x_image)), 'dec_image': -10 * np.ones(len(y_image))}]
kwargs_upper_ps = [{'ra_image': 10* np.ones(len(x_image)), 'dec_image': 10 * np.ones(len(y_image))}]

fixed_cosmo = {}
kwargs_cosmo_init = {'D_dt': 1000}
kwargs_cosmo_sigma = {'D_dt': 100}
kwargs_lower_cosmo = {'D_dt': 200}
kwargs_upper_cosmo = {'D_dt': 3000}

cosmo_params = [kwargs_cosmo_init, kwargs_cosmo_sigma, fixed_cosmo, kwargs_lower_cosmo, kwargs_upper_cosmo]

ps_params = [kwargs_ps_init, kwargs_ps_sigma, fixed_ps, kwargs_lower_ps, kwargs_upper_ps]

if flag_inference == 'inference_no_ellipt' or flag_mock =='no_ellipt':
    lens_model_list = ['PEMD', 'SHEAR', 'NFW']

kwargs_model = {'lens_model_list': lens_model_list,
                 'lens_light_model_list': lens_light_model_list,
                 'source_light_model_list': source_model_list,
                'point_source_model_list': point_source_list
                 }
# Init, upper and lower bound values for all parameters of the model
kwargs_params = {'lens_model': lens_params,
                'source_model': source_params,
                'lens_light_model': lens_light_params,
                'point_source_model': ps_params,
                'special': cosmo_params}

# numerical options and fitting sequences
num_source_model = len(source_model_list)
kwargs_constraints = {'joint_source_with_point_source': [[0, 0]],
                      'num_point_source_list': [4],
                      'solver_type': 'PROFILE_SHEAR',  # 'PROFILE', 'PROFILE_SHEAR', 'ELLIPSE', 'CENTER'
                      'Ddt_sampling': True,
                      }

cosmo_reference = FlatLambdaCDM(H0=70, Om0=0.3, Ob0=0.)
lens_cosmo_reference = LensCosmo(z_lens = z_lens, z_source = z_source, cosmo = cosmo_reference)
Ddt_reference = lens_cosmo_reference.ddt
print('H0 prior values', 70 * Ddt_reference / 3000, 70 * Ddt_reference / 1000, 70 * Ddt_reference / 200)
# Defining the prior
Delta_c = 178*(Om*(1+z_lens)**3/(Om*(1+z_lens)**3 + OL))**0.45
r_halo_sigma = 16
index_nfw = 2
cvir_true = 4.26

from scipy import interpolate

class LikelihoodAddition(object):
    import numpy as np
    def __init__(self):
        pass
    def __call__(self, kwargs_lens=None, kwargs_source=None, kwargs_lens_light=None, kwargs_ps=None, kwargs_special=None, kwargs_extinction=None):
        return self.logL_addition(kwargs_lens, kwargs_source, kwargs_lens_light, kwargs_ps, kwargs_special, kwargs_extinction)
    def logL_addition(self, kwargs_lens, kwargs_source=None, kwargs_lens_light=None, kwargs_ps=None, kwargs_special=None, kwargs_extinction=None):
        """
        a definition taking as arguments (kwargs_lens, kwargs_source, kwargs_lens_light, kwargs_ps, kwargs_special, kwargs_extinction)
                and returns a logL (punishing) value.
        """
        Ddt_sampled =  kwargs_special['D_dt']
        # Ddt_reference = 1668.213101294793
        h0_sampled = 70 * Ddt_reference / Ddt_sampled

        # Functions for kinematics likelihood, from Wong et al 2011
        rho_c = 3 * h0_sampled**2 * (Om*(1+z_lens)**3+OL) /(8*np.pi*G_const)*1e12*pc2meter / m_sun # Critical density in msun/Mpc^3
        if flag_center =='no_priors':
            return np.log(h0_sampled/Ddt_sampled)
        if flag_kin=='_no_kin_':
            c_sampled = cvir_true
        else:
            cosmo_current = FlatLambdaCDM(H0= h0_sampled, Om0=Om)
            lens_cosmo_current = LensCosmo(z_lens= z_lens, z_source= z_source, cosmo= cosmo_current)
            sigma_crit = lens_cosmo_current.sigma_crit 
            dd = lens_cosmo_current.dd 
            Rs_sampled, alpha_Rs_sampled = kwargs_lens[index_nfw]['Rs'], kwargs_lens[index_nfw]['alpha_Rs']

            rho0 = alpha_Rs_sampled / (4 * Rs_sampled ** 2 * (1 + np.log(1. / 2.)))/arcsec2rad*sigma_crit/dd
            c_array = np.linspace(1, 20, 100)
            f_c_array = c_array**3/(np.log(1+c_array) -c_array/(1+c_array)) - 3*rho0/(Delta_c*rho_c)
            c_f_interp = interpolate.InterpolatedUnivariateSpline(f_c_array, c_array, w=None, bbox=[None, None], k=3)
            c_sampled = c_f_interp(0)
            cvir_minus, cvir_plus = 10**(np.log10(c_sampled) -0.14), 10**(np.log10(c_sampled) +0.14)
            M_pivot = 1e12 /(h0_sampled/100)# msun
            Rs_plus = (10**(0.971)*(4*np.pi* rho0*(np.log(1+cvir_minus) -cvir_minus/(1+cvir_minus))/M_pivot)**(-0.094)/cvir_minus)**(1/(0.094*3))
            Rs_min = (10**(0.971)*(4*np.pi* rho0*(np.log(1+cvir_plus) -cvir_plus/(1+cvir_plus))/M_pivot)**(-0.094)/cvir_plus)**(1/(0.094*3))
        if c_sampled > 20: # Hard cutoff for very unphysical situations
            return -np.inf
        elif flag_kin =='_cprior_' and (Rs_sampled < Rs_min/dd/arcsec2rad or Rs_sampled > Rs_plus/dd/arcsec2rad): 
            return -np.inf
        else:
            if flag_kin =='_no_kin_':
                sigma_sampled = 0
            else:
                sigma_sampled = 4*np.pi*rho0*G_const*(Rs_sampled*arcsec2rad*dd)**2/3/c_sampled*(np.log(1+c_sampled) - c_sampled/(1+c_sampled))
                sigma_sampled = np.sqrt(sigma_sampled*m_sun/(1e6*pc2meter))/1e3
            logL =  np.log(h0_sampled/Ddt_sampled) - (sigma_halo_mean - sigma_sampled)**2/(2*sigma_halo_err**2)
            if flag_center == 'flat_prior':
                return logL
            else:
                x_sampled, y_sampled = kwargs_lens[index_nfw]['center_x'], kwargs_lens[index_nfw]['center_y']
                r_sampled2 = (x_sampled - x_halo_prior)**2 + (y_sampled- y_halo_prior)**2
                return logL - r_sampled2/(2*r_halo_sigma**2)
logL_addition = LikelihoodAddition()
# Recall to add here the logL_addition for correct prior
kwargs_likelihood = {'check_bounds': True,
                     'force_no_add_image': False,
                     'source_marg': False,
                     'image_position_uncertainty': 0.004,
                     'check_matched_source_position': True,
                     'source_position_tolerance': 0.001,
                     'source_position_sigma': 0.01,
                     'time_delay_likelihood': True,
                     'custom_logL_addition': logL_addition
                             }

print(lens_model_list, lens_params, kwargs_model)
print('sigma r', r_halo_sigma)
# kwargs_data contains the image array
image_band = [kwargs_data, kwargs_psf, kwargs_numerics]
multi_band_list = [image_band]
kwargs_data_joint = {'multi_band_list': multi_band_list, 'multi_band_type': 'multi-linear',
                    'time_delays_measured': dt_measured,
                    'time_delays_uncertainties': dt_sigma,}
#  print('Check values, x_prior', x_halo_prior, 'y_prior', y_halo_prior, 'x_source', source_x, 'y_source', source_y)

In [ ]:
from lenstronomy.Workflow.fitting_sequence import FittingSequence

if run_sim == True:
    fitting_seq = FittingSequence(kwargs_data_joint, kwargs_model, kwargs_constraints, kwargs_likelihood, kwargs_params, mpi=mpi)
    # Do before the PSO to reach a good starting value for MCMC
    if start_from_backup == False:
        fitting_kwargs_list = [#['PSO', {'sigma_scale': 1., 'n_particles': 200, 'n_iterations': 200}],
                ['MCMC', {'sampler_type':'ZEUS','n_burn': 800, 'n_run': 500, 'walkerRatio': 20, 'sigma_scale': .2,
                    'backend_filename': backup_filename, 'progress': True,
                    #  'kwargs_zeus': {'ncheck': 10, 'autocorrelation_callback': True}}],
                    'moves': zeus.moves.DifferentialMove()}]
                ,['MCMC', {'sampler_type':'ZEUS','n_burn': 700, 'n_run': 1000, 'walkerRatio': 20, 'sigma_scale': .2,
                    'backend_filename': backup_filename, 'progress': True,
                    'moves': zeus.moves.GlobalMove()}]
        ]
    else:
        with h5py.File(backup_filename2, "r") as hf:
            samples_mcmc = np.copy(hf['samples'])
            #logprob_samples = np.copy(hf['logprob'])
            initpos = samples_mcmc[-1]
        fitting_kwargs_list = [['MCMC', {'sampler_type':'ZEUS','n_burn': 0, 'n_run': 3000, 'walkerRatio': 20, 'sigma_scale': .2, 'init_samples':initpos,
                    'backend_filename': backup_filename, 'start_from_backend': start_from_backup, 'progress': True, 'moves': zeus.moves.GlobalMove()}]]

    start_time = time.time()
    chain_list = fitting_seq.fit_sequence(fitting_kwargs_list)
    kwargs_result = fitting_seq.best_fit()

    file_name = name_flag+'mock_result_store.pkl'
    filedata = open(file_name, 'wb')
    pickle.dump(kwargs_result, filedata)
    filedata.close()

    file_name = name_flag+'mock_results_chain.pkl'
    filedata = open(file_name, 'wb')
    pickle.dump(chain_list, filedata)
    filedata.close()
    end_time = time.time()

    print(end_time - start_time, 'total time needed for computation')
else:
    file_name = name_flag+'mock_result_store.pkl'
    filedata = open(file_name, 'rb')
    kwargs_result = pickle.load(filedata)
    filedata.close()

    file_name = name_flag+'mock_results_chain.pkl'
    filedata = open(file_name, 'rb')
    chain_list = pickle.load(filedata)
    filedata.close()

print('Final parameters given by MCMC: ', kwargs_result)

In [ ]:
import corner
from lenstronomy.Plots import chain_plot
from lenstronomy.Plots.model_plot import ModelPlot

# Figures of mock reconstruction
modelPlot = ModelPlot(multi_band_list, kwargs_model, kwargs_result, arrow_size=0.02, cmap_string="gist_heat")
print('Number of data', modelPlot._imageModel.num_data_evaluate)
f, axes = modelPlot.plot_main()
#f.savefig(name_flag+'Plot_main.pdf')
f, axes = modelPlot.plot_separate()
#f.savefig(name_flag+'Plot_separate.pdf')
f, axes = modelPlot.plot_subtract_from_data_all()
#f.savefig(name_flag+'Plot_subtract.pdf')

# Plot the MonteCarlo
for i in range(len(chain_list)):
    chain_plot.plot_chain_list(chain_list, i)
#  chain_plot.plt.show()
chain_plot.plt.savefig(name_flag+'chainPlot.pdf')

# This to make a range for the cornerplot, single numbers are to make a fraction
# of the whole range, cutting bounds (1 means don't cut anything)
#  range_ = [1, 1, 1, 1, 1, 1, 1, 1]
if flag_mock == 'no_ellipt' or flag_inference == 'inference_no_ellipt':
    range_ = [1,1, 1, 1, 1, (0.0005,0.004), 1, 1, 1, 1, 1, 1, 1]
    labels_new = [r"$\theta_{\rm E}$", r"$ \gamma $", r"$ e_1 $",r"$ e_2 $",r"$ R_{\rm s} $", r"$ \tilde{\rho}_0 $", r"$ c^{\rm nfw}_x $", r"$ c^{\rm nfw}_y $", r"$ c^{\rm source}_x $", r"$ c^{\rm source}_y $", r"$ \gamma^{\rm shear}_1 $",r"$ \gamma^{\rm shear}_2 $",  r"$ H_0 $"]
    truths = [theta_E_true, gamma_true, e1true, e2true, Rs_angle, tilde_rho_true, x_nfw, y_nfw, source_x, source_y, gamma1_true_shear, gamma2_true_shear, 67.4]
else:
    range_ = [1,1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,1 ,1]
    labels_new = [r"$\theta_{\rm E}$", r"$ \gamma $", r"$ e_1 $",r"$ e_2 $",r"$ R_{\rm s} $", r"$ \tilde{\rho}_0 $", r"$ c^{\rm nfw}_x $", r"$ c^{\rm nfw}_y $", r"$ c^{\rm source}_x $", r"$ c^{\rm source}_y $", r"$ \gamma^{\rm shear}_1 $",r"$ \gamma^{\rm shear}_2 $", r"$ e^{\rm nfw}_1 $", r"$ e^{\rm nfw}_2 $", r"$ H_0 $"]
    truths = [theta_E_true, gamma_true, e1true, e2true, Rs_angle, tilde_rho_true, x_nfw, y_nfw, source_x, source_y, gamma1_true_shear, gamma2_true_shear, e1_nfw_true, e2_nfw_true, 67.4]

# title_kwargs and label_kwarks can be used to change fontsizes
kwargs_corner = {'bins': 20, 'plot_datapoints': False, 'show_titles': True, 'title_kwargs': dict(fontsize = 18),
                 'label_kwargs': dict(fontsize=20), 'smooth': 0.5, 'levels': [0.68,0.95],
                 'fill_contours': True, 'alpha': 0.8  #, 'range': range_
                 }

# Set to False if you want to just change customization things
if start_from_backup == True:
    chain_list_index = 0
else:
    chain_list_index = 1

sampler_type, samples_mcmc, param_mcmc, dist_mcmc  = chain_list[chain_list_index]

print("number of non-linear parameters in the MCMC process: ", len(param_mcmc))
print("parameters in order: ", param_mcmc)
print("number of evaluations in the MCMC process: ", np.shape(samples_mcmc)[0])

# import the parameter handling class
from lenstronomy.Sampling.parameters import Param
# make instance of parameter class with given model options, constraints and fixed parameters #

param = Param(kwargs_model, fixed_lens, fixed_source, fixed_lens_light, fixed_ps, fixed_cosmo,
              kwargs_lens_init=kwargs_result['kwargs_lens'], **kwargs_constraints)
# the number of non-linear parameters and their names #
num_param, param_list = param.num_param()

mcmc_new_list = []

In [ ]:
if reprocess_corner == True:
    for i in range(len(samples_mcmc)):
        # transform the parameter position of the MCMC chain in a lenstronomy convention with keyword arguments #
        kwargs_result = param.args2kwargs(samples_mcmc[i])
        theta_E = kwargs_result['kwargs_lens'][0]['theta_E']
        gamma = kwargs_result['kwargs_lens'][0]['gamma']
        e1, e2 = kwargs_result['kwargs_lens'][0]['e1'], kwargs_result['kwargs_lens'][0]['e2']
        center_source_x, center_source_y = kwargs_result['kwargs_source'][0]['center_x'], kwargs_result['kwargs_source'][0]['center_y']
        center_nfw_x, center_nfw_y = kwargs_result['kwargs_lens'][index_nfw]['center_x'], kwargs_result['kwargs_lens'][index_nfw]['center_y']

        ddt_mcmc = kwargs_result['kwargs_special']['D_dt']
        h0 = 70 * Ddt_reference / ddt_mcmc
        cosmo_current = FlatLambdaCDM(H0= h0, Om0=0.3)
        lens_cosmo_current = LensCosmo(z_lens= z_lens, z_source= z_source, cosmo= cosmo_current)
        Rs_sampled, alpha_Rs_sampled = kwargs_result['kwargs_lens'][index_nfw]['Rs'], kwargs_result['kwargs_lens'][index_nfw]['alpha_Rs']
        #  rho0, Rs, c, r200, M200 = lens_cosmo_current.nfw_angle2physical(Rs_angle=Rs_sampled, alpha_Rs=alpha_Rs_sampled)
        rho_tilde = alpha_Rs_sampled / (4 * Rs_sampled ** 2 * (1 + np.log(1. / 2.)))

        gamma1_shear, gamma2_shear = kwargs_result['kwargs_lens'][1]['gamma1'], kwargs_result['kwargs_lens'][1]['gamma2']
        if flag_mock == 'no_ellipt' or flag_inference == 'inference_no_ellipt':
            mcmc_new_list.append([theta_E, gamma, e1, e2, Rs_sampled, rho_tilde, center_nfw_x, center_nfw_y, center_source_x, center_source_y, gamma1_shear, gamma2_shear, h0])
        else:
            e1_nfw, e2_nfw = kwargs_result['kwargs_lens'][index_nfw]['e1'], kwargs_result['kwargs_lens'][index_nfw]['e2']
            mcmc_new_list.append([theta_E, gamma, e1, e2, Rs_sampled, rho_tilde, center_nfw_x, center_nfw_y, center_source_x, center_source_y, gamma1_shear, gamma2_shear, e1_nfw, e2_nfw, h0])

    file_name = name_flag+'mock_corner.h5'
    h5file = h5py.File(file_name, 'w')
    h5file.create_dataset("dataset_mock", data=mcmc_new_list)
    h5file.close()
else:
    file_name = name_flag+'mock_corner.h5'
    h5file = h5py.File(file_name, 'r')
    mcmc_new_list = h5file['dataset_mock'][:]
    h5file.close()

In [ ]:
plot = corner.corner(np.array(mcmc_new_list), labels=labels_new, truths=truths, range = range_, **kwargs_corner)
#plot.savefig(name_flag+'corner_plot.pdf')
if flag_mock == 'ellipt' and flag_inference=='same':
    mcmc_new_list = np.array(mcmc_new_list)[:,(6,7,14)]
else:
    mcmc_new_list = np.array(mcmc_new_list)[:,(6,7,12)]
range_ = [1, 1, 1]
labels_new = [r"$ c^{\rm nfw}_x $", r"$ c^{\rm nfw}_y $", r"$ H_0 $"]
truths = [x_nfw, y_nfw, 67.4]

plot = corner.corner(np.array(mcmc_new_list), labels=labels_new, truths=truths, range = range_, **kwargs_corner)